## Análise Estatística, Correlação e Regressão

### 4.2 Inferência Estatística
## Preparação dos dados

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.stattools import durbin_watson

# Configuração dos gráficos matplotlib 
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 13
plt.rcParams['axes.titlesize'] = 15
plt.rcParams['xtick.labelsize'] = 11
plt.rcParams['ytick.labelsize'] = 11
plt.rcParams['legend.fontsize'] = 11
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3
plt.rcParams['axes.axisbelow'] = True  # Grid atrás dos dados
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False

# Paleta de cores personalizada
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', 
          '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

# Estilo personalizado
plt.style.use('seaborn-v0_8-whitegrid')

print("================ ANÁLISE DE DADOS ================")
print("================ TRABALHO PRÁTICO ================")
print()

# Carregar os dados do csv
df = pd.read_csv('AIRPOL_data.csv', sep=';')

# Remover colunas desnecessárias
df = df.drop(columns=[col for col in df.columns if "Unnamed" in col])

# Converter colunas para numérico (substituindo vírgulas por pontos)
numeric_columns = ['Air_Pollution_Average[ug/m3]', 'Value', 'Populated_Area[km2]', 'Affected_Population']
for col in numeric_columns:
    if col in df.columns:
        df[col] = df[col].astype(str).str.replace(',', '.').astype(float)
        


# Verificar os tipos de dados após a conversão
print("Tipos de dados das colunas após conversão:")
print(df.dtypes)
print("\n" + "="*70 + "\n")


#### 4.2.1 - Seleção de Amostra Aleatória
Selecionamos aleatoriamente 50 registos dos níveis médios de poluição atmosférica em Portugal para análise estatística.

In [ ]:
# 4.2.1 Selecionar os valores relativos a Portugal
print("4.2.1. Amostra aleatória de Portugal")
df_portugal = df[df['Country'] == 'Portugal']

# Verificar se há dados suficientes para Portugal
if len(df_portugal) < 50:
    print(f"Aviso: Portugal tem apenas {len(df_portugal)} registros, o que é menor que os 50 necessários.")
    print("Será feita amostragem com reposição.")
    np.random.seed(42)  # Para reprodutibilidade
    airpol_pt = df_portugal['Air_Pollution_Average[ug/m3]'].sample(n=50, replace=True, random_state=42)
else:
    # Amostra aleatória de 50 registros de Portugal
    np.random.seed(42)  # Para reprodutibilidade
    airpol_pt = df_portugal['Air_Pollution_Average[ug/m3]'].sample(n=50, random_state=42)

print(f"Tamanho da amostra de Portugal: {len(airpol_pt)}")
print(f"Estatísticas descritivas da amostra:")
print(airpol_pt.describe().round(4).to_string())
print()

#### 4.2.2 - Teste de Hipótese: Portugal vs Albânia
Realizamos um teste t para comparar se a média dos níveis de poluição atmosférica em Portugal é inferior à da Albânia. O teste assume a seguinte hipótese:

- **H₀**: A média dos níveis de poluição em Portugal é maior ou igual à da Albânia.
- **H₁**: A média dos níveis de poluição em Portugal é menor do que a da Albânia.

A estatística do teste t é dada por:
$$
t = \frac{\bar{X}_1 - \bar{X}_2}{\sqrt{\frac{s_1^2}{n_1} + \frac{s_2^2}{n_2}}}
$$
onde:
- $$\bar{X}_1, \bar{X}_2$$ são as médias das amostras,
- $$s_1^2, s_2^2$$ são as variâncias das amostras,
- $$n_1, n_2$$ são os tamanhos das amostras.

In [ ]:
# 4.2.2 Testar se o nível médio de poluição atmosférica em Portugal é inferior ao da Albânia
print("4.2.2 Comparação entre Portugal e Albânia")
df_albania = df[df['Country'] == 'Albania']
airpol_albania = df_albania['Air_Pollution_Average[ug/m3]']

# Calcular as estatísticas
mean_pt = airpol_pt.mean()
mean_albania = airpol_albania.mean()
std_pt = airpol_pt.std(ddof=1)
n_pt = len(airpol_pt)

# Teste t unilateral (H0: média_PT >= média_Albania, H1: média_PT < média_Albania)
t_stat = (mean_pt - mean_albania) / (std_pt / np.sqrt(n_pt))
p_value = stats.t.cdf(t_stat, n_pt-1)  # Unilateral à esquerda

print(f"Média de poluição em Portugal (amostra): {mean_pt:.4f}")
print(f"Média de poluição na Albânia: {mean_albania:.4f}")
print(f"Estatística t: {t_stat:.4f}")
print(f"Valor-p (unicaudal): {p_value:.4f}")

if p_value < 0.05:
    print("Conclusão: Há evidência estatística para afirmar que o nível médio de poluição atmosférica")
    print("em Portugal é inferior ao nível médio na Albânia, com um nível de significância de 5%.")
else:
    print("Conclusão: Não há evidência estatística suficiente para afirmar que o nível médio de poluição")
    print("atmosférica em Portugal é inferior ao nível médio na Albânia, com um nível de significância de 5%.")
print()

#### 4.2.3 - Teste de Hipótese: Espanha vs França
Selecionamos aleatoriamente duas amostras de 20 registos para Espanha e França e realizamos um teste t para verificar se há diferença significativa entre os níveis médios de poluição atmosférica nesses países.

- **H₀**: Não há diferença significativa entre os níveis médios de poluição atmosférica em Espanha e França.
- **H₁**: Existe uma diferença significativa entre os níveis médios de poluição atmosférica em Espanha e França.


In [ ]:
# 4.2.3 Comparar níveis de poluição entre Espanha e França
print("4.2.3. Comparação entre Espanha e França")
df_espanha = df[df['Country'] == 'Spain']
df_franca = df[df['Country'] == 'France']

# Verificar se há dados suficientes
min_sample_size = min(len(df_espanha), len(df_franca))
if min_sample_size < 20:
    print(f"Aviso: Um dos países tem menos de 20 registros. Usando {min_sample_size} registros para cada.")
    sample_size = min_sample_size
else:
    sample_size = 20

# Amostras aleatórias de registros de cada país
np.random.seed(42)  # Para reprodutibilidade
airpol_espanha = df_espanha['Air_Pollution_Average[ug/m3]'].sample(n=sample_size, random_state=42)
airpol_franca = df_franca['Air_Pollution_Average[ug/m3]'].sample(n=sample_size, random_state=42)

# Verificação de variâncias iguais usando teste de Levene
_, p_levene = stats.levene(airpol_espanha, airpol_franca)
equal_var = p_levene > 0.05  # Se p > 0.05, assumimos variâncias iguais

# Teste t para amostras independentes
t_stat_ef, p_value_ef = stats.ttest_ind(airpol_espanha, airpol_franca, equal_var=equal_var)

print(f"Média de poluição em Espanha (amostra): {airpol_espanha.mean():.4f}")
print(f"Média de poluição em França (amostra): {airpol_franca.mean():.4f}")
print(f"Teste de Levene para igualdade de variâncias p-value: {p_levene:.4f}")
print(f"Estatística t: {t_stat_ef:.4f}")
print(f"Valor-p: {p_value_ef:.4f}")

if p_value_ef < 0.05:
    print("Conclusão: Rejeitamos H0. Há evidência estatística para afirmar que existe diferença")
else:
    print("Conclusão: Não rejeitamos H0. Não há evidência estatística suficiente para afirmar que existe diferença")
    
print("significativa entre os níveis médios de poluição atmosférica entre Espanha e França, com um nível de significância de 5%.")
print()

#### 4.2.4 - ANOVA: Portugal, Albânia, Espanha e França
Para comparar os níveis médios de poluição atmosférica entre quatro países, utilizamos uma ANOVA (Análise de Variância). A estatística F do teste ANOVA é definida por:
$$
F = \frac{\text{Variância entre grupos}}{\text{Variância dentro dos grupos}}
$$
Se o teste indicar diferenças significativas, realizaremos uma análise post-hoc para identificar quais países apresentam diferenças significativas entre si.

In [ ]:
 #4.2.4 Comparar níveis de poluição entre Portugal, Albânia, Espanha e França
print("4.2.4. Comparação entre Portugal, Albânia, Espanha e França")

# Verificar se há dados suficientes para todos os países
countries = ['Portugal', 'Albania', 'Spain', 'France']
country_sizes = {country: len(df[df['Country'] == country]) for country in countries}
min_size = min(country_sizes.values())

if min_size < 20:
    print(f"Aviso: O país com menos dados tem apenas {min_size} registros.")
    anova_sample_size = min_size
else:
    anova_sample_size = 20

print(f"Tamanho da amostra para cada país: {anova_sample_size}")

# Criar amostras para cada país
samples = {}
for country in countries:
    country_data = df[df['Country'] == country]['Air_Pollution_Average[ug/m3]']
    samples[country] = country_data.sample(n=anova_sample_size, replace=(len(country_data) < anova_sample_size), random_state=42)

# Criar DataFrame para ANOVA
data_for_anova = []
for country, sample in samples.items():
    for value in sample:
        data_for_anova.append({'Country': country, 'AirPollution': value})

df_anova = pd.DataFrame(data_for_anova)

# ANOVA
model = sm.formula.ols('AirPollution ~ Country', data=df_anova).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

print("\nANOVA para comparação entre os 4 países:")
print(anova_table.round(4).to_string())

if anova_table.iloc[0]['PR(>F)'] < 0.05:
    print("\nConclusão: Rejeitamos H0. Há evidência estatística para afirmar que existe pelo menos")
    print("uma diferença significativa entre os níveis médios de poluição atmosférica nos quatro países,")
    print("com um nível de significância de 5%. Prosseguimos com o teste post-hoc.")
    
    # Teste post-hoc de Tukey
    tukey = pairwise_tukeyhsd(endog=df_anova['AirPollution'], groups=df_anova['Country'], alpha=0.05)
    
    print("\nTeste post-hoc de Tukey:")
    print(tukey)
    
    # Interpretação do teste de Tukey
    print("\nInterpretação do teste post-hoc de Tukey:")
    tukey_df = pd.DataFrame(data=tukey._results_table.data[1:], columns=tukey._results_table.data[0])
    
    significant_pairs = tukey_df[tukey_df['reject'] == True]
    if len(significant_pairs) > 0:
        print("Pares de países com diferenças significativas:")
        for _, row in significant_pairs.iterrows():
            print(f"- {row['group1']} e {row['group2']}: diferença de {row['meandiff']:.4f}")
    else:
        print("Nenhum par de países apresentou diferença significativa no teste post-hoc.")
else:
    print("\nConclusão: Não rejeitamos H0. Não há evidência estatística suficiente para afirmar que")
    print("existe diferença significativa entre os níveis médios de poluição atmosférica nos quatro países,")
    print("com um nível de significância de 5%.")

# Visualização das diferenças entre os grupos - apenas boxplot com estética melhorada
plt.close('all')
plt.figure(figsize=(10, 6))

# Usar paleta de cores personalizadas para os países
palette = {country: colors[i] for i, country in enumerate(countries)}
sns.boxplot(x='Country', y='AirPollution', data=df_anova, hue='Country', palette=palette, 
            width=0.6, linewidth=1.5, legend=False)

plt.title('Níveis de poluição atmosférica por país', fontweight='bold', fontsize=14)
plt.xlabel('País', fontsize=12)
plt.ylabel('Nível médio de poluição (ug/m³)', fontsize=12)
plt.grid(True, axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

print("\n" + "="*70 + "\n")
